# Create Pipeline in combo w/ GridsearchCV

## Install packages on machine
uncomment to install on machine

In [1]:
# import sys
#!{sys.executable} -m pip install numpy
#!{sys.executable} -m pip install pandas
#!{sys.executable} -m pip install scikit-learn
#!{sys.executable} -m pip install xgboost
#!{sys.executable} -m pip install lightgbm
#!{sys.executable} -m pip install imbalanced-learn

In [1]:
import pandas as pd
import numpy as np
from imblearn.pipeline import Pipeline
from imblearn import FunctionSampler
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import PCA, KernelPCA, SparsePCA
from sklearn.cluster import FeatureAgglomeration
from sklearn.neighbors import LocalOutlierFactor, KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer, QuantileTransformer
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import SimpleImputer, KNNImputer, MissingIndicator, IterativeImputer
from sklearn.feature_selection import VarianceThreshold, SelectKBest, mutual_info_regression, \
                                      f_regression, SelectFromModel, GenericUnivariateSelect, \
                                      SequentialFeatureSelector
from sklearn.svm import SVR
from sklearn.ensemble import IsolationForest, RandomForestRegressor
from sklearn.svm import LinearSVC
from sklearn.linear_model import LinearRegression, LogisticRegression, HuberRegressor, \
                                 RANSACRegressor, BayesianRidge, ARDRegression
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import r2_score

from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV, RandomizedSearchCV

pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_rows', 500) 

In [2]:
pd.set_option('display.max_columns', 100) 

In [3]:
path = './data/'

In [4]:
df_train = pd.read_csv(path+'X_train.csv')
df_test = pd.read_csv(path+'X_test.csv')
df_y = pd.read_csv(path+'y_train.csv')

In [5]:
X_train = df_train.drop('id', axis=1)
y_train = df_y.drop('id', axis=1).values.ravel()

In [6]:
df_train.head()

,id,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,...,x782,x783,x784,x785,x786,x787,x788,x789,x790,x791,x792,x793,x794,x795,x796,x797,x798,x799,x800,x801,x802,x803,x804,x805,x806,x807,x808,x809,x810,x811,x812,x813,x814,x815,x816,x817,x818,x819,x820,x821,x822,x823,x824,x825,x826,x827,x828,x829,x830,x831
0,0.00,10.89,832442.81,20585.54,1028.37,1163779.65,9.20,597900.48,NaN,1144294.32,785176.20,38042.01,106.64,67475.90,944696.31,895.09,10.82,155995.45,10.11,98.78,90.20,107.18,1232961.03,2538.05,1038292.48,10.45,1530.00,10403.21,101.97,9902.07,10.68,1.97,108.10,10.49,985.74,139494.79,5555.41,10079.67,NaN,1199.95,14709.22,10095.17,2.34,2.40,1096980.87,101012.23,81625.69,10860.45,68128.14,1096216.56,...,10.89,16053.67,42.13,2457.08,2.33,1003485.69,9739.38,1.84,11863.86,9046.05,2.59,107835.47,1050202.85,961.00,102.73,11230.08,1132.88,10955.85,2.36,1079014.02,10.27,309702.42,1074.95,10083.17,106208.12,102.07,119.51,NaN,1077.84,NaN,87766.03,1857.07,97.00,NaN,1011.96,89.18,107577.83,2.14,-3163.22,1057.89,1024198.18,-855.55,12176.07,10.65,10.92,1220.07,8.57,1036263.30,85338.56,103088.66
1,1.00,11.51,832442.90,NaN,1012.62,1028910.54,10.91,597900.46,8127.02,1099166.21,785176.26,50949.42,NaN,93179.78,926043.43,1159.07,10.40,NaN,10.37,104.57,88.72,102.70,NaN,2033.00,832197.06,10.50,2119.10,10700.28,101.35,10414.08,10.86,1.72,102.05,10.07,880.97,NaN,4784.22,10968.23,NaN,937.23,12616.74,10115.08,1.68,1.61,1089746.56,103534.19,102066.62,11275.87,66250.71,1049788.49,...,10.54,23841.49,55.58,2295.39,1.85,1097167.82,10792.88,1.74,9787.79,10628.02,1.80,104811.16,1091030.19,1052.82,82.46,10487.07,695.41,10985.26,1.89,1072501.91,8.18,557053.46,1060.35,10458.07,108798.12,100.21,NaN,95559.70,1028.51,212339.93,99387.41,1818.09,101.16,1073.75,1017.27,NaN,NaN,1.75,-2661.69,1025.25,1086805.85,-787.40,10493.10,10.59,9.46,917.09,10.23,1007163.22,95695.02,105161.11
2,2.00,11.05,832442.90,20585.51,1003.95,923175.65,9.21,597900.43,10738.09,1027862.70,785176.22,47259.32,103.37,69377.20,828900.31,NaN,10.67,155995.37,10.44,94.99,101.70,100.03,NaN,2138.05,679221.19,10.97,1926.04,10633.37,107.43,12163.03,10.58,2.34,101.25,10.78,NaN,139494.78,4317.86,10628.20,10381.02,977.00,13070.57,10167.33,2.48,2.42,1030810.23,102961.09,97557.78,9597.02,70022.03,1034560.65,...,10.41,17141.62,84.68,1471.91,2.17,1070666.36,9933.96,2.11,9188.11,12841.02,2.93,101814.68,1001865.32,NaN,96.29,11122.09,862.89,10308.21,2.41,1048036.02,NaN,396496.09,1004.19,10249.49,NaN,106.15,94.67,91827.99,1061.75,212339.94,108348.19,1895.07,NaN,1015.71,903.52,100.03,108205.36,2.42,-2671.97,1039.02,1018532.66,-907.00,10959.52,10.77,10.34,637.03,10.71,1019955.44,80253.30,104177.05
3,3.00,11.64,NaN,NaN,1004.67,945946.09,9.55,597900.45,13524.10,1168144.27,785176.25,53507.05,NaN,85953.18,903459.38,NaN,10.31,155995.46,10.92,116.30,125.95,100.95,850443.90,2618.09,NaN,11.00,2402.02,10786.77,103.73,17640.00,10.87,2.42,106.30,10.31,1028.88,139494.77,5728.03,10139.01,10450.82,1074.22,20564.62,NaN,2.50,2.60,1080037.60,100576.67,85236.97,9101.54,93777.35,1098240.87,...,10.67,23914.06,51.06,884.02,2.62,1081970.77,11303.67,2.03,10657.74,15391.02,2.96,107142.21,NaN,980.71,98.96,13292.07,NaN,10864.96,2.65,1043134.14,11.02,496861.94,1058.96,10797.09,109698.61,108.19,98.05,87665.33,1060.99,212339.97,95656.50,2390.01,NaN,NaN,836.11,86.56,105872.41,2.62,-4593.30,1095.51,1047017.25,-1011.74,16845.31,10.48,10.59,1114.07,10.32,1085441.78,NaN,102746.52
4,4.00,10.41,832442.83,20585.56,NaN,995718.19,8.42,597900.42,12894.07,1063198.89,785176.19,52954.78,109.73,90224.27,847909.41,1115.52,10.85,155995.37,10.50,109.52,106.70,95.62,916477.65,2462.02,1031983.85,10.55,1784.06,10131.65,107.34,16711.05,10.96,2.09,100.33,10.75,916.49,139494.82,5535.49,10160.92,10369.45,1041.76,22497.00,10721.60,2.53,2.49,1071558.84,103708.34,101219.75,9559.19,87380.28,1085843.00,...,10.90,21434.91,1366.02,1832.26,2.23,1

## Parameters

In [7]:
scoring = 'r2'
cv = 5
n_jobs = -1 # if not 1 gridsearch does not print verbose output/progress
refit = True

## Helper functions for outlier detection wrapper

In [8]:
def outlier_rejection_isolationforest(X, y, **kwargs):
    """This will be our function used to resample our dataset."""
    y_pred = IsolationForest(**kwargs).fit_predict(X)
    return X[y_pred == 1], y[y_pred == 1]

def outlier_rejection_LOF(X, y, **kwargs):
    """This will be our function used to resample our dataset."""
    y_pred = LocalOutlierFactor(**kwargs).fit_predict(X)
    return X[y_pred == 1], y[y_pred == 1]

In [14]:
search_space = [ 
  # {'scaler': [RobustScaler()],
  #  'imputer': #[KNNImputer(n_neighbors=200)],
  #             [IterativeImputer(n_nearest_features=80, add_indicator=True)],
  #             # IterativeImputer(n_nearest_features=80, add_indicator=False)],
  #  'variance': [VarianceThreshold(0)],
  #  'selector': [SelectKBest()],
  #  'selector__k': [200],
  #  'selector__score_func': [f_regression],
  #  'dimreduction': [KernelPCA(n_jobs=-1)],
  #  'dimreduction__n_components': [150],
  #  'regressor': [SVR()],
  #  'regressor__C': [200],
  #  'regressor__epsilon': [0.5],
  #  'outliers_1': [FunctionSampler(func=outlier_rejection_isolationforest, validate=True, \
  #                                 kw_args={'max_samples': 100, 'contamination': 0.4})],
  #  'outliers_2': [FunctionSampler(func=outlier_rejection_LOF, validate=True, \
  #                                 kw_args={'n_neighbors': 80, 'contamination': 0.4, 'n_jobs':-1})],
  # },
 
   {'scaler': [RobustScaler()],
    'imputer': #[KNNImputer(n_neighbors=200)],
               [IterativeImputer(n_nearest_features=80, add_indicator=True)],
               # IterativeImputer(n_nearest_features=80, add_indicator=False)],
    'variance': [VarianceThreshold(0)],
    'selector': [SelectKBest()],
    'selector__k': [200],
    'selector__score_func': [f_regression],
    'dimreduction': [KernelPCA(n_jobs=-1)],
    'dimreduction__n_components': [150],
    'regressor': [SVR()],
    'regressor__C': [200],
    'regressor__epsilon': [0.5],
    'outliers_1': [FunctionSampler(func=outlier_rejection_isolationforest, validate=True, \
                                   kw_args={'n_estimators': 600, 'max_samples': 100})],
    'outliers_2': [FunctionSampler(func=outlier_rejection_LOF, validate=True, \
                                   kw_args={'n_neighbors': 80})],  
   },

    {'scaler': [RobustScaler()],
    'imputer': #[KNNImputer(n_neighbors=200)],
               [IterativeImputer(n_nearest_features=80, add_indicator=True, initial_strategy='median')],
               # IterativeImputer(n_nearest_features=80, add_indicator=False)],
    'variance': [VarianceThreshold(0)],
    'selector': [SelectKBest()],
    'selector__k': [200],
    'selector__score_func': [f_regression],
    'dimreduction': [KernelPCA(n_jobs=-1)],
    'dimreduction__n_components': [150],
    'regressor': [SVR()],
    'regressor__C': [200],
    'regressor__epsilon': [0.5],
    'outliers_1': [FunctionSampler(func=outlier_rejection_isolationforest, validate=True, \
                                   kw_args={})],
    'outliers_2': [FunctionSampler(func=outlier_rejection_LOF, validate=True, \
                                   kw_args={})],     
    },

   {'scaler': [RobustScaler()],
    'imputer': #[KNNImputer(n_neighbors=200)],
               [IterativeImputer(n_nearest_features=80, add_indicator=True)],
               # IterativeImputer(n_nearest_features=80, add_indicator=False)],
    'variance': [VarianceThreshold(0)],
    'selector': [SelectKBest()],
    'selector__k': [200],
    'selector__score_func': [f_regression],
    'dimreduction': [KernelPCA(n_jobs=-1)],
    'dimreduction__n_components': [150],
    'regressor': [SVR()],
    'regressor__C': [200],
    'regressor__epsilon': [0.5],
    'outliers_1': [FunctionSampler(func=outlier_rejection_isolationforest, validate=True, \
                                   kw_args={'max_samples': 300, 'contamination': 0.1})],
    'outliers_2': [FunctionSampler(func=outlier_rejection_LOF, validate=True, \
                                   kw_args={'n_neighbors': 80, 'contamination': 0.1, 'n_jobs':-1})],
   },

  # {'scaler': [RobustScaler()],
  # 'imputer': #[KNNImputer(n_neighbors=200)],
  #            [IterativeImputer(n_nearest_features=80, add_indicator=True)],
  #            # IterativeImputer(n_nearest_features=80, add_indicator=False)],
  # 'variance': [VarianceThreshold(0)],
  # 'selector': [SelectKBest()],
  # 'selector__k': [200],
  # 'selector__score_func': [f_regression],
  # 'dimreduction': [KernelPCA(n_jobs=-1)],
  # 'dimreduction__n_components': [150],
  # 'regressor': [SVR()],
  # 'regressor__C': [200],
  # 'regressor__epsilon': [0.5],
  # 'outliers_1': [FunctionSampler(func=outlier_rejection_isolationforest, validate=True, \
  #                                kw_args={'max_samples': 100, 'contamination': 0.3})],
  # 'outliers_2': [FunctionSampler(func=outlier_rejection_LOF, validate=True, \
  #                                kw_args={'n_neighbors': 40, 'contamination': 0.3, 'n_jobs':-1})],
  #},
    ## kNN imputer - SVR regressor
    #{'scaler': [RobustScaler(), QuantileTransformer()],
    # 'imputer': [KNNImputer()],
    # 'imputer__n_neighbors': [20, 100, 300],
    # 'imputer__weights': ['distance', 'uniform'],
    # 'variance': [VarianceThreshold()],
    # 'selector': [GenericUnivariateSelect(score_func=f_regression, mode='k_best', param=200)],
    # 'dimreduction': [SparsePCA(n_jobs=-1), KernelPCA(n_jobs=-1)],
    # 'dimreduction__n_components': [0.95, 0.85, 200, None],
    # 'outliers': [FunctionSampler(func=outlier_rejection_isolationforest, validate=False),
    #              FunctionSampler(func=outlier_rejection_LOF, validate=False)],
    # 'regressor': [SVR()],
    # 'regressor__C': [1e-4, 1, 200],
    # 'regressor__epsilon': [1e-4, 1, 10],
    #},
    
  # ## Forward selection
  # {'scaler': [RobustScaler(), QuantileTransformer()],
  #  'imputer': [KNNImputer()],
  #  'imputer__n_neighbors': [80, 200],
  #  'dimreduction': [KernelPCA(n_jobs=-1)],
  #  'dimreduction__n_components': [0.95, 0.85, 200, None],
  #  'selector': [SequentialFeatureSelector(estimator=KNeighborsClassifier(n_neighbors=3), \
  #                                        direction='forward', n_jobs=-1)],
  #  'selector__n_features_to_select': [50, 100, 200],
  #  'outliers': [FunctionSampler(func=outlier_rejection_isolationforest, validate=False),
  #               FunctionSampler(func=outlier_rejection_LOF, validate=False)],
  #  'regressor': [RandomForestRegressor(n_estimators=600)],
  # },
  # ## Select from Model
  # {'scaler': [RobustScaler(), QuantileTransformer()],
  # 'imputer': [KNNImputer()],
  # 'imputer__n_neighbors': [80, 200],
  # 'variance': [VarianceThreshold()],
  # 'dimreduction': [KernelPCA(n_jobs=-1)],
  # 'dimreduction__n_components': [0.95, 0.85, 200, None],
  # 'selector': [
  #   SelectFromModel(estimator=LogisticRegression()),
  #   SelectFromModel(estimator=LinearSVC(penalty="l1"))],
  # 'outliers': [FunctionSampler(func=outlier_rejection_isolationforest, validate=False),
  #           FunctionSampler(func=outlier_rejection_LOF, validate=False)],
  # 'regressor': [RandomForestRegressor(n_estimators=600)]
  # }
]

## Design Pipeline

In [35]:
pipeline = Pipeline([
    ('scaler', RobustScaler()),
    ('imputer', IterativeImputer(n_nearest_features=80, add_indicator=True, initial_strategy='median')), # KNN with add indicator
    ('variance', VarianceThreshold(0)), # allows NaN in the input
    ('selector', SelectKBest(k=200)),
    ('dimreduction', KernelPCA(n_components=150)),
    ('outliers_1', FunctionSampler(func=outlier_rejection_isolationforest, validate=True, kw_args={'max_samples': 100, 'n_estimators': 600})),
    ('outliers_2', FunctionSampler(func=outlier_rejection_LOF, validate=True, kw_args={'n_neighbors': 80})),
    ('regressor', SVR(C=100, epsilon=0.5)),
])

In [ ]:
estimator = pipeline.fit(X_train, y_train)

In [17]:
estimator

In [18]:
X_test = df_test.drop('id', axis=1)

In [21]:
## Fitted with pipeline - no gridsearch
y_pred = estimator.predict(X_test)
df_y_pred = pd.DataFrame(y_pred)
df_y_pred['id'] = df_test.id
df_y_pred.columns = ['y', 'id']
df_y_pred = df_y_pred[['id', 'y']]
df_y_pred.to_csv('y_pred_gamma_auto.csv', index=False)

In [15]:
df_y_pred

,id,y
0,0.00,69.62
1,1.00,74.06
2,2.00,70.14
3,3.00,70.46
4,4.00,70.24
...,...,...
771,771.00,56.21
772,772.00,72.97
773,773.00,75.16
774,774.00,66.01


In [19]:
grid = GridSearchCV(estimator=pipeline, param_grid=search_space, cv=cv, verbose=10,
                    n_jobs=1, scoring=scoring, return_train_score=False, refit=refit)

In [20]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 1/5; 1/3] START dimreduction=KernelPCA(n_jobs=-1), dimreduction__n_components=150, imputer=IterativeImputer(add_indicator=True, n_nearest_features=80), outliers_1=FunctionSampler(func=<function outlier_rejection_isolationforest at 0x7facfc5e9d30>,
                kw_args={'max_samples': 100, 'n_estimators': 600}), outliers_2=FunctionSampler(func=<function outlier_rejection_LOF at 0x7facfc5e9b80>,
                kw_args={'n_neighbors': 80}), regressor=SVR(), regressor__C=200, regressor__epsilon=0.5, scaler=RobustScaler(), selector=SelectKBest(), selector__k=200, selector__score_func=<function f_regression at 0x7facabad94c0>, variance=VarianceThreshold(threshold=0)


/home/rapwag01/.virtualenvs/vethaml/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


[CV 1/5; 1/3] END dimreduction=KernelPCA(n_jobs=-1), dimreduction__n_components=150, imputer=IterativeImputer(add_indicator=True, n_nearest_features=80), outliers_1=FunctionSampler(func=<function outlier_rejection_isolationforest at 0x7facfc5e9d30>,
                kw_args={'max_samples': 100, 'n_estimators': 600}), outliers_2=FunctionSampler(func=<function outlier_rejection_LOF at 0x7facfc5e9b80>,
                kw_args={'n_neighbors': 80}), regressor=SVR(), regressor__C=200, regressor__epsilon=0.5, scaler=RobustScaler(), selector=SelectKBest(), selector__k=200, selector__score_func=<function f_regression at 0x7facabad94c0>, variance=VarianceThreshold(threshold=0);, score=0.745 total time= 7.6min
[CV 2/5; 1/3] START dimreduction=KernelPCA(n_jobs=-1), dimreduction__n_components=150, imputer=IterativeImputer(add_indicator=True, n_nearest_features=80), outliers_1=FunctionSampler(func=<function outlier_rejection_isolationforest at 0x7facfc5e9d30>,
                kw_args={'max_samples': 

/home/rapwag01/.virtualenvs/vethaml/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


[CV 2/5; 1/3] END dimreduction=KernelPCA(n_jobs=-1), dimreduction__n_components=150, imputer=IterativeImputer(add_indicator=True, n_nearest_features=80), outliers_1=FunctionSampler(func=<function outlier_rejection_isolationforest at 0x7facfc5e9d30>,
                kw_args={'max_samples': 100, 'n_estimators': 600}), outliers_2=FunctionSampler(func=<function outlier_rejection_LOF at 0x7facfc5e9b80>,
                kw_args={'n_neighbors': 80}), regressor=SVR(), regressor__C=200, regressor__epsilon=0.5, scaler=RobustScaler(), selector=SelectKBest(), selector__k=200, selector__score_func=<function f_regression at 0x7facabad94c0>, variance=VarianceThreshold(threshold=0);, score=0.645 total time= 6.5min
[CV 3/5; 1/3] START dimreduction=KernelPCA(n_jobs=-1), dimreduction__n_components=150, imputer=IterativeImputer(add_indicator=True, n_nearest_features=80), outliers_1=FunctionSampler(func=<function outlier_rejection_isolationforest at 0x7facfc5e9d30>,
                kw_args={'max_samples': 

...

[CV 4/5; 3/3] END dimreduction=KernelPCA(n_jobs=-1), dimreduction__n_components=150, imputer=IterativeImputer(add_indicator=True, n_nearest_features=80), outliers_1=FunctionSampler(func=<function outlier_rejection_isolationforest at 0x7facfc5e9d30>,
                kw_args={'contamination': 0.1, 'max_samples': 300}), outliers_2=FunctionSampler(func=<function outlier_rejection_LOF at 0x7facfc5e9b80>,
                kw_args={'contamination': 0.1, 'n_jobs': -1, 'n_neighbors': 80}), regressor=SVR(), regressor__C=200, regressor__epsilon=0.5, scaler=RobustScaler(), selector=SelectKBest(), selector__k=200, selector__score_func=<function f_regression at 0x7facabad94c0>, variance=VarianceThreshold(threshold=0);, score=0.529 total time= 4.9min
[CV 5/5; 3/3] START dimreduction=KernelPCA(n_jobs=-1), dimreduction__n_components=150, imputer=IterativeImputer(add_indicator=True, n_nearest_features=80), outliers_1=FunctionSampler(func=<function outlier_rejection_isolationforest at 0x7facfc5e9d30>,
   

/home/rapwag01/.virtualenvs/vethaml/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


[CV 5/5; 3/3] END dimreduction=KernelPCA(n_jobs=-1), dimreduction__n_components=150, imputer=IterativeImputer(add_indicator=True, n_nearest_features=80), outliers_1=FunctionSampler(func=<function outlier_rejection_isolationforest at 0x7facfc5e9d30>,
                kw_args={'contamination': 0.1, 'max_samples': 300}), outliers_2=FunctionSampler(func=<function outlier_rejection_LOF at 0x7facfc5e9b80>,
                kw_args={'contamination': 0.1, 'n_jobs': -1, 'n_neighbors': 80}), regressor=SVR(), regressor__C=200, regressor__epsilon=0.5, scaler=RobustScaler(), selector=SelectKBest(), selector__k=200, selector__score_func=<function f_regression at 0x7facabad94c0>, variance=VarianceThreshold(threshold=0);, score=0.604 total time=17.4min


/home/rapwag01/.virtualenvs/vethaml/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', RobustScaler()),
                                       ('imputer',
                                        IterativeImputer(add_indicator=True,
                                                         n_nearest_features=80)),
                                       ('variance',
                                        VarianceThreshold(threshold=0)),
                                       ('selector', SelectKBest(k=200)),
                                       ('dimreduction',
                                        KernelPCA(n_components=150)),
                                       ('outliers_1',
                                        FunctionSampler(func=<function outlier_rejection_isolationforest at 0x7facfc5...
                          'outliers_2': [FunctionSampler(func=<function outlier_rejection_LOF at 0x7facfc5e9b80>,
                                                         kw_args={'contamination': 0.1,
   

In [21]:
df_results = pd.DataFrame(grid.cv_results_)

In [22]:
df_results.to_csv('cv_results_')

In [32]:
df_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_dimreduction,param_dimreduction__n_components,param_imputer,param_outliers_1,param_outliers_2,param_regressor,param_regressor__C,param_regressor__epsilon,param_scaler,param_selector,param_selector__k,param_selector__score_func,param_variance,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,419.84,50.14,2.33,0.46,KernelPCA(n_jobs=-1),150,"IterativeImputer(add_indicator=True, n_nearest...",FunctionSampler(func=<function outlier_rejecti...,FunctionSampler(func=<function outlier_rejecti...,SVR(),200,0.50,RobustScaler(),SelectKBest(),200,<function f_regression at 0x7facabad94c0>,VarianceThreshold(threshold=0),"{'dimreduction': KernelPCA(n_jobs=-1), 'dimred...",0.75,0.65,0.56,0.56,0.66,0.64,0.07,2
1,308.94,45.95,2.04,0.44,"KernelPCA(n_components=150, n_jobs=-1)",150,"IterativeImputer(add_indicator=True, initial_s...",FunctionSampler(func=<function outlier_rejecti...,FunctionSampler(func=<function outlier_rejecti...,"SVR(C=200, epsilon=0.5)",200,0.50,RobustScaler(),"SelectKBest(k=200, score_func=<function f_regr...",200,<function f_regression at 0x7facabad94c0>,VarianceThreshold(threshold=0),"{'dimreduction': KernelPCA(n_components=150, n...",0.70,0.67,0.56,0.57,0.68,0.64,0.06,1
2,427.25,307.61,2.22,0.83,KernelPCA(n_jobs=-1),150,"IterativeImputer(add_indicator=True, n_nearest...",FunctionSampler(func=<function outlier_rejecti...,FunctionSampler(func=<function outlier_rejecti...,SVR(),200,0.50,RobustScaler(),SelectKBest(),200,<function f_regression at 0x7facabad94c0>,VarianceThreshold(threshold=0),"{'dimreduction': KernelPCA(n_jobs=-1), 'dimred...",0.67,0.60,0.51,0.53,0.60,0.58,0.06,3


In [34]:
df_results[['param_outliers_1']].values

array([[FunctionSampler(func=<function outlier_rejection_isolationforest at 0x7facfc5e9d30>,
                        kw_args={'max_samples': 100, 'n_estimators': 600})                  ],
       [FunctionSampler(func=<function outlier_rejection_isolationforest at 0x7facfc5e9d30>,
                        kw_args={})                                                         ],
       [FunctionSampler(func=<function outlier_rejection_isolationforest at 0x7facfc5e9d30>,
                        kw_args={'contamination': 0.1, 'max_samples': 300})                 ]],
      dtype=object)

In [24]:
best_score = grid.best_score_
best_estimator = grid.best_estimator_

In [25]:
best_score

0.6359176809604398

In [26]:
 grid.best_estimator_

Pipeline(steps=[('scaler', RobustScaler()),
                ('imputer',
                 IterativeImputer(add_indicator=True, initial_strategy='median',
                                  n_nearest_features=80)),
                ('variance', VarianceThreshold(threshold=0)),
                ('selector',
                 SelectKBest(k=200,
                             score_func=<function f_regression at 0x7facabad94c0>)),
                ('dimreduction', KernelPCA(n_components=150, n_jobs=-1)),
                ('outliers_1',
                 FunctionSampler(func=<function outlier_rejection_isolationforest at 0x7facfc5e9d30>,
                                 kw_args={})),
                ('outliers_2',
                 FunctionSampler(func=<function outlier_rejection_LOF at 0x7facfc5e9b80>,
                                 kw_args={})),
                ('regressor', SVR(C=200, epsilon=0.5))])

In [27]:
y_pred = best_estimator.predict(X_test)
df_y_pred = pd.DataFrame(y_pred)

In [28]:
df_y_pred['id'] = df_test.id
df_y_pred.columns = ['y', 'id']
df_y_pred = df_y_pred[['id', 'y']]

In [29]:
df_y_pred.to_csv('y_pred_iterative_imp80_indTrue_initmedian.csv', index=False)